Building likelihood maps for a set of representative images.

> TODO: use images from dataset and superimpose the bounding boxes on them


In [ ]:
import retinoto_py as fovea
subset_factor = 50
args = fovea.Params(do_fovea=True, batch_size=1, subset_factor=subset_factor)
args

Params(image_size=224, do_mask=False, do_fovea=True, rs_min=0.0, rs_max=-7.5, padding_mode='zeros', seed=2025, batch_size=1, num_workers=1, in_memory=False, model_name='convnext_base', num_epochs=41, subset_factor=50, lr=2e-07, delta1=0.2, delta2=0.007, weight_decay=0.003, label_smoothing=0.05, do_full_training=False, shuffle=True, verbose=False)

In [2]:
dataset = 'bbox'
VAL_DATA_DIR = args.DATAROOT / f'Imagenet_{dataset}' / 'val'
val_dataset = fovea.get_dataset(args, VAL_DATA_DIR, do_full_preprocess=False)
val_loader = fovea.get_loader(args, val_dataset)


In [3]:
model_filename = args.data_cache /  f'32_fovea_model_name={args.model_name}_dataset={dataset}.pth'
model = fovea.load_model(args, model_filename=model_filename)
model_filename

PosixPath('cached_data/32_fovea_model_name=convnext_base_dataset=bbox.pth')

In [ ]:
from torchvision.transforms import v2 as transforms
npy_filename = args.data_cache / '42_likelihood_maps.npy'

# %rm {npy_filename}  # FORCING RECOMPUTE

In [ ]:
resolution = (100, 100)
resolution = (30, 30)
resolution = (20, 20)
size_ratio = .4

if npy_filename.exists():
    results = fovea.np.load(npy_filename)
else:
    n_dataset = len(val_dataset)
    with fovea.torch.no_grad():
        results = fovea.np.empty((resolution[0], resolution[1], n_dataset))
        for i_batch, (image, true_idx) in fovea.tqdm(enumerate(val_loader), total=n_dataset):
            image, true_idx = image.to(args.device), true_idx.to(args.device)

            pos_H, pos_W, probas = fovea.compute_likelihood_map(args, model, image.squeeze(0), size_ratio=size_ratio, resolution=resolution)
            results[:, :, i_batch] = probas[:, true_idx].cpu().numpy().reshape(resolution)
        fovea.np.save(npy_filename, results)        

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
# fig, ax = fovea.plt.subplots()
# contour = ax.contourf(results.mean(axis=-1))
# fig.colorbar(contour, ax=ax)  # Add colorbar
# ax.axis('square')
# fig.set_facecolor(color='white')


In [ ]:
from torchvision.transforms import v2 as transforms

resolution = (100, 100)
resolution = (30, 30)
resolution = (25, 25)
size_ratio = .4

for do_fovea in [True, False]:
    for dataset in fovea.all_datasets:
        args = fovea.Params(do_mask=not(do_fovea), do_fovea=do_fovea, 
                            batch_size=1, subset_factor=subset_factor)
        VAL_DATA_DIR = args.DATAROOT / f'Imagenet_{dataset}' / 'val'
        val_dataset = fovea.get_dataset(args, VAL_DATA_DIR, do_full_preprocess=False)
        val_loader = fovea.get_loader(args, val_dataset)

        model_filename = args.data_cache /  f'32_fovea_model_name={args.model_name}_dataset=bbox.pth'
        model_filename = args.data_cache /  f'20_model_name={args.model_name}_dataset=bbox.pth'
        model = fovea.load_model(args, model_filename=model_filename)

        npy_filename = args.data_cache / f'42_likelihood_maps_{dataset=}_{do_fovea=}.npy'
        # %rm {npy_filename}  # FORCING RECOMPUTE
        if npy_filename.exists():
            results = fovea.np.load(npy_filename)
        else:
            n_dataset = len(val_dataset)
            with fovea.torch.no_grad():
                results = fovea.np.empty((resolution[0], resolution[1], n_dataset))
                for i_batch, (image, true_idx) in fovea.tqdm(enumerate(val_loader), total=n_dataset):
                    image, true_idx = image.to(args.device), true_idx.to(args.device)
# crop_size = min(image.shape[1], image.shape[2])
            # image = transforms.CenterCrop(crop_size)(image)
                    pos_H, pos_W, probas = fovea.compute_likelihood_map(args, model, image.squeeze(0), size_ratio=size_ratio, resolution=resolution)
                    results[:, :, i_batch] = probas[:, true_idx].cpu().numpy().reshape(resolution)
                fovea.np.save(npy_filename, results)        
        results.min(), results.max()

In [ ]:
results.max(axis=-1).shape

In [ ]:
results.shape

In [ ]:
i_image = 42
proba = results[:, :, :, i_image]
proba.min(), proba.max()

In [ ]:
fig, ax = fovea.plt.subplots()
contour = ax.contourf(results.mean(axis=-1))
fig.colorbar(contour, ax=ax)  # Add colorbar
ax.axis('square')
fig.set_facecolor(color='white')


In [ ]:
results.max(axis=(0, 1, 2)).shape

In [ ]:
accuracy = results.max(axis=(0, 1, 2))
fig, ax = fovea.plt.subplots()
ax.hist(accuracy)


In [ ]:
dataset = fovea.get_dataset(args, VAL_DATA_DIR)
len(dataset)


In [ ]:
from torchvision import datasets

dataset = datasets.ImageFolder(root=VAL_DATA_DIR)
len(dataset), len(dataset.targets), dataset.targets[51:53]

In [ ]:

# Get all targets

import numpy as np 
targets = np.array(dataset.targets)
targets[:53]

In [ ]:

# Get unique classes and their indices
classes, counts = np.unique(targets, return_counts=True)
classes, counts


In [ ]:
len(classes)

In [ ]:
subset_factor = 50
# Calculate the number of samples per class in the subset
n_per_class = min(counts) // subset_factor
n_per_class

In [ ]:
cls = 0
cls_indices = np.where(targets == cls)[0]
cls_indices

In [ ]:

# Sample indices for each class
subset_indices = []
for cls in classes:
    cls_indices = np.where(targets == cls)[0]
    np.random.shuffle(cls_indices)
    subset_indices.extend(cls_indices[:n_per_class])


# Shuffle the subset indices
# np.random.shuffle(subset_indices)

len(subset_indices)

In [ ]:
from torch.utils.data import Dataset, DataLoader, Subset
sub_dataset = Subset(dataset, subset_indices)


In [ ]:
len(sub_dataset)

In [ ]:
sub_dataset